In [1]:
import cv2
import torch
from ultralytics import YOLO
from collections import defaultdict

In [2]:
# --------------------------------------------------------------------------
# 1. 설정 (Setup)
# --------------------------------------------------------------------------

# ⭐️⭐️⭐️ 여기에 학습시킨 '여드름 부분 탐지' 모델의 .pt 파일 경로를 입력하세요 ⭐️⭐️⭐️
# 예시: 'acne_lesion_detection/yolo11n_lesion_detection_run/weights/best.pt'
MODEL_PATH = 'acne_lesion_detection/yolo11n_lesion_detection_run/weights/best.pt'

# GPU 또는 CPU 설정
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {DEVICE}")

Using device: cuda


In [3]:
# 모델 로드
try:
    model = YOLO(MODEL_PATH)
    model.to(DEVICE)
    print("모델 로드 성공!")
except Exception as e:
    print(f"모델 로드 실패: {e}")
    print("MODEL_PATH 변수에 정확한 모델 가중치(.pt) 파일 경로를 입력했는지 확인하세요.")
    exit()

# 웹캠 초기화
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("웹캠을 열 수 없습니다.")
    exit()

모델 로드 성공!


In [4]:
# --------------------------------------------------------------------------
# 2. 심각도 판별 함수 (Severity Logic)
# --------------------------------------------------------------------------
def get_acne_severity(lesion_counts):
    """
    탐지된 여드름 병변의 종류와 개수를 바탕으로 심각도를 판별하는 함수.
    (의학적 기준을 단순화한 규칙 기반 로직)
    """
    total_lesions = sum(lesion_counts.values())
    
    # 심각한 병변 (결절, 낭종 등)의 개수 확인
    # 모델 클래스 이름을 소문자로 변환하여 비교 (예: 'nodules', 'cyst')
    severe_lesion_count = lesion_counts.get('nodules', 0) + \
                          lesion_counts.get('acne fulminans', 0) # 전격성 여드름 등

    # 규칙 기반 심각도 분류
    if total_lesions == 0:
        return "정상 (Normal)", (0, 255, 0)  # 녹색
    elif severe_lesion_count >= 3 or total_lesions > 30:
        return "심각 (Severe)", (0, 0, 255)  # 빨간색
    elif severe_lesion_count >= 1 or total_lesions > 15:
        return "중등도 (Moderate)", (0, 165, 255)  # 주황색
    else:
        return "경증 (Mild)", (0, 255, 255)  # 노란색

In [5]:
# --------------------------------------------------------------------------
# 3. 메인 루프 (Main Loop)
# --------------------------------------------------------------------------
while True:
    # 웹캠에서 프레임 읽기
    ret, frame = cap.read()
    if not ret:
        print("프레임을 읽을 수 없습니다. 종료합니다.")
        break

    # 모델 추론 수행
    results = model(frame, verbose=False) # verbose=False로 설정하여 터미널 로그 최소화

    # 탐지된 객체 정보 처리
    lesion_counts = defaultdict(int)
    
    # results[0]에 현재 프레임의 탐지 결과가 들어있음
    for box in results[0].boxes:
        # 클래스 ID와 신뢰도 점수 추출
        class_id = int(box.cls[0])
        confidence = float(box.conf[0])
        
        # 신뢰도가 0.5 이상인 경우에만 처리
        if confidence > 0.5:
            # 클래스 이름 가져오기 (소문자로 변환하여 일관성 유지)
            class_name = model.names[class_id].lower()
            lesion_counts[class_name] += 1
            
            # 바운딩 박스 좌표
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            
            # 바운딩 박스 그리기
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
            
            # 클래스 이름과 신뢰도 표시
            label = f"{class_name.capitalize()}: {confidence:.2f}"
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    # 심각도 판별
    severity, color = get_acne_severity(lesion_counts)

    # 화면에 정보 표시
    y_offset = 30
    cv2.putText(frame, "Acne Detection & Severity Analysis", (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
    y_offset += 30
    
    # 심각도 표시
    cv2.putText(frame, f"Severity: {severity}", (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
    y_offset += 30

    # 각 여드름 개수 표시
    for name, count in lesion_counts.items():
        if count > 0:
            cv2.putText(frame, f"- {name.capitalize()}: {count}", (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            y_offset += 25

    # 결과 화면 출력
    cv2.imshow('Webcam Acne Detection', frame)

    # 'q' 키를 누르면 루프 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

In [6]:
# --------------------------------------------------------------------------
# 4. 종료 (Cleanup)
# --------------------------------------------------------------------------
cap.release()
cv2.destroyAllWindows()
print("프로그램을 종료합니다.")

프로그램을 종료합니다.
